In [23]:
import pandas
import random
import numpy as np
import itertools

with open('valid_guesses.csv') as f:
    guesses = f.read().splitlines()
with open('valid_solutions.csv') as f:
    solutions = f.read().splitlines()


In [123]:
class LetterPos:
    def __init__(self, letter, pos, gpos, perfect):
        self.letter = letter
        self.pos = pos
        self.gpos = gpos
        self.perfect = perfect

class GameState:
    def __init__(self):
        self.guesses = []
        self.solution = random.choice(solutions)
        self.correct_letters = []
        self.incorrect_letters = []
        self.turn = 0
        self.state = True

    def output_turn(self):
        if self.correct_letters[self.turn] is not None:
            for i in range(len(self.correct_letters[self.turn])):
                print(self.correct_letters[self.turn][i].letter)
                print(self.correct_letters[self.turn][i].pos)
                print(self.correct_letters[self.turn][i].perfect)

    def pretty_output(self):
        guess_result = list("⬛⬛⬛⬛⬛")
        if self.correct_letters[self.turn] is not None:
            for i in range(len(self.correct_letters[self.turn])):
                guess_result[self.correct_letters[self.turn][i].gpos] = "🟩" if self.correct_letters[self.turn][i].perfect else "🟨"
        print(''.join(guess_result))

    def globalHints(self):
        return list(itertools.chain(*self.correct_letters))

    def nextTurn(self):
        if self.turn < 5:
            self.turn += 1
        else:
            self.state = False
    
    def endgame(self):
        print("You won in : " +  str(self.turn + 1))
        self.state = False
     

In [124]:
def compare(game, guess, solution):
    #compare the two words
    print(guess)
    guess_letters = list(guess)
    solution_letters = list(solution)
    
    if(guess_letters == solution_letters):
        print("🟩🟩🟩🟩🟩")
        game.endgame()
        return
    
    
    correct_letters = []
    incorrect_letters = []
    #check for letter matches
    for i in range(5):
        for j in range(5):
            if(solution_letters[i] == guess_letters[j]):
                guess_grade = LetterPos(guess_letters[j], i, j, (i == j))
                correct_letters.append(guess_grade)
                break

    #check for letter misses
    for i in range(5):
        if(guess_letters[i] not in solution_letters):
            incorrect_letters.append(guess_letters[i])
            
    game.incorrect_letters = list(set(game.incorrect_letters + incorrect_letters))

    #edge cases
    #check for repeating LetterPos
    #if two LetterPos classes have the same letter and pos pop one
    # for i in range(len(correct_letters)):
    #     for j in range(len(correct_letters)):
    #         if(correct_letters[i].pos == correct_letters[j].pos and i != j):
    #             correct_letters.pop(j)

    #win condition
    #check if the guess is perfect
    # if(len(correct_letters) == 5):
    #     isPerfect = True
    #     for i in range(len(correct_letters)):
    #         if(correct_letters[i].perfect == False):
    #             isPerfect = False
    #     if(isPerfect):


    
    game.correct_letters.append(correct_letters)

    

In [121]:
#game loop
def wordlegame():
    game = GameState()
    #randomly select a word from the list solutions
    print(game.solution)
    
    while(game.state):
        # guess = random_guess()

        # guess = elimination_guess(game)

        guess = kinda_smart(game)


            #compare the guess to the solution
        compare(game, guess, game.solution)
        if(game.state):
            game.pretty_output()
            #print(game.incorrect_letters)
            game.nextTurn()


In [125]:
wordlegame()

guide
force
⬛⬛⬛⬛🟩
['c', 'f', 'r', 'o']
stave
⬛⬛⬛⬛🟩
['o', 'v', 't', 'a', 'c', 'f', 's', 'r']
libel
⬛🟨⬛🟨⬛
['o', 'v', 't', 'a', 'l', 'c', 'f', 'b', 's', 'r']
['e', 'e', 'i', 'e']
index
🟨⬛🟨🟨⬛
['o', 'v', 't', 'x', 'l', 'a', 'c', 'n', 'f', 'b', 's', 'r']
['e', 'e', 'i', 'e', 'i', 'd', 'e']
guide
🟩🟩🟩🟩🟩
You won in : 5


In [48]:
def random_guess():
    guess = random.choice(guesses)
    return guess

In [7]:
def elimination_guess(game):
    possible_words = []


    possible_words = [ele for ele in solutions if all(ch not in ele for ch in game.incorrect_letters)]

    guess = random.choice(possible_words)
    
    return guess

In [46]:
def kinda_smart(game):
    #for the first 3 turns, guess words without incorrect letters
    if(game.turn < 3):
        guess = elimination_guess(game)
        return guess
    #for the next 3 guess words with the correct letters and without incorrect letters
    else:
    #process the correct letter objects, words must contain those letters (no positions) and no incorrect letters
        correct_chars = []
        correct_letters = game.globalHints()
        for i in range(len(correct_letters)):
            correct_chars.append(correct_letters[i].letter)
        #print(correct_chars)

        possible_words = [ele for ele in solutions if all(ch not in ele for ch in game.incorrect_letters)]
        possible_solutions = [ele for ele in possible_words if all(ch in ele for ch in correct_chars)]

        guess = random.choice(possible_solutions)
        
        return guess


    